In [ ]:
import os
import re
import pymongo
import yaml

# Open credential file for database
with open(os.path.join(os.getenv('HOME'), 'db.profile')) as f:
    db_profile = f.read()

# Connect to the MongoDB as an admin (i.e. has read AND write access)
db = pymongo.MongoClient(host="mongodb05.nersc.gov", 
                             username="bella_sf_admin",
                             password=re.findall('SF_DB_ADMIN_PASSWORD=(.+)', db_profile)[0],  
                             authSource="bella_sf")["bella_sf"]

In [ ]:
# List the different collections
# (one collection per BELLA experimental setup, e.g. IP2, A-cave, HTU)
for collection in db.list_collections():
    print(collection['name'])

In [ ]:
collection = db['models']

In [ ]:
experiment = 'acave'
model_type = 'NN'

In [ ]:
collection.delete_many({})

# Upload model

In [ ]:
path = f'/global/cfs/cdirs/m558/superfacility/ml/saved_models/{model_type}_training/'

In [ ]:
with open(os.path.join(path, experiment+'.yml')) as f:
    yaml_file_content = f.read()

In [ ]:
document = {
    'experiment': experiment,
    'model_type': model_type,
    'yaml_file_content': yaml_file_content
}
model_info = yaml.safe_load(yaml_file_content)
for filename in [ model_info['model'] ] + model_info['input_transformers'] + model_info['output_transformers']:
    with open(os.path.join(path, filename), 'rb') as f:
        document[filename] = f.read()
collection.insert_one( document )

In [ ]:
for document in collection.find({}):
    print(document)

# Download and test model

In [ ]:
# TODO: check that there is only one document
for document in collection.find({'experiment': experiment, 'model_type': model_type}):
    yaml_file_content = document['yaml_file_content']
    with open(experiment+'.yml', 'w') as f:
        f.write(yaml_file_content)
    for filename in [ model_info['model'] ] + model_info['input_transformers'] + model_info['output_transformers']:
        print(filename)
        with open(os.path.join(filename), 'wb') as f:
            f.write( document[filename] )

In [ ]:
from lume_model.models.torch_model import TorchModel

In [ ]:
TorchModel('acave.yml')